<a href="https://colab.research.google.com/github/ananyamalik09/huggingface-finetuning/blob/main/huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Preprocess GLUE MRPC dataset using dynamic padding


In [1]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.3 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, DataCollatorWithPadding

In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset

In [5]:
raw_datasets = load_dataset("glue","mrpc")

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [6]:
checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [7]:
def tokenize_function(examples):
  return tokenizer(examples["sentence1"],examples["sentence2"], truncation=True)

In [8]:
tokenized_datasets= raw_datasets.map(tokenize_function, batched = True)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [9]:
tokenized_datasets= tokenized_datasets.remove_columns(["sentence1","sentence2","idx"])
tokenized_datasets= tokenized_datasets.rename_column("label","labels")
tokenized_datasets.set_format("torch")

In [10]:
data_collator = DataCollatorWithPadding(tokenizer)

##Create DataLoader

In [11]:
from torch.utils.data import DataLoader

In [12]:
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=8,collate_fn=data_collator)

In [13]:
eval_dataloader = DataLoader(tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator)

In [14]:
for batch in train_dataloader:
  break
print({k: v.shape for k,v in batch.items()})

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([8]), 'input_ids': torch.Size([8, 89]), 'token_type_ids': torch.Size([8, 89]), 'attention_mask': torch.Size([8, 89])}


##Create model

In [15]:
from transformers import AutoModelForSequenceClassification

In [16]:
checkpoint = "bert-base-cased"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(0.8176, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


---
**----NOTE---**


The double asterisks (**) before the batch variable in the function call are used to unpack the dictionary elements as keyword arguments. This means that each key-value pair in the batch dictionary will be treated as a separate keyword argument to the model function.

For example, if the batch dictionary looks like this:

```
batch = {
    'input_ids': [1, 2, 3, 4],
    'attention_mask': [1, 1, 1, 1],
    'token_type_ids': [0, 0, 1, 1],
}
```
The function call model(**batch) would be equivalent to calling the model function with the following arguments:


```
model(input_ids=[1, 2, 3, 4], attention_mask=[1, 1, 1, 1], token_type_ids=[0, 0, 1, 1])

```
**----END---**


---

In [18]:
from transformers import AdamW

In [19]:
optimizer = AdamW(model.parameters(), lr = 5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


---
**----NOTE---**


AdamW = optimiser

Optimiser is responsible for carring out training updates to model weights

**----END---**

---


In [20]:
optimizer.zero_grad()
loss = outputs.loss
loss.backward()
optimizer.step()

In [21]:
#why are gradients zeroed after optimiser step? otherwise next step is added to gradient we computed
optimizer.zero_grad()

---
**----NOTE---**

A gradient represents the rate of change of the loss function concerning each parameter in the neural network. In simple terms, it tells us in which direction and how much we should adjust the parameters to reduce the loss.

When you call the loss.backward() function, it calculates the gradients for all the model's trainable parameters with respect to the loss. It's like finding the slopes of a hilly landscape that represents our loss function.

Now, let's talk about why we need to reset the gradients using optimizer.zero_grad() before every backward pass. Consider you have a bucket, and you want to measure the water flow from a faucet using this bucket. You need to empty the bucket each time you measure to get the accurate water flow for the current time interval. Similarly, in the training process, we want to measure the gradients only for the current batch and not accumulate the gradients from all the previous batches.

If we don't reset the gradients and perform multiple backward passes without clearing them, the gradients would keep accumulating across batches. This would be like measuring the water flow but not emptying the bucket between measurements. As a result, the accumulated gradients would mislead the optimization algorithm, leading to unpredictable and often undesirable updates to the model's parameters. In the worst case, it could even prevent the model from converging to a good solution.

Therefore, by resetting the gradients before each backward pass with optimizer.zero_grad(), we ensure that we start each batch with a clean slate. It allows us to accurately measure the gradients for the current batch only and perform meaningful updates to the model's parameters based on that specific batch's contribution to the overall loss function.

In summary, resetting the gradients is essential to maintain the integrity of the optimization process and enable the neural network to learn effectively during training. It ensures that each batch's information is properly considered and that the model converges to a better solution over time.

**----END---**

---


In [22]:
from transformers import get_scheduler

In [23]:
num_epochs = 3
num_training_steps = num_epochs*len(train_dataloader)
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0,num_training_steps= num_training_steps)


---
**----NOTE---**

Learning rate scheduler: to progressively decay the learning rate to zero

Reasons for using a learning rate scheduler are:

1. **Smooth convergence**: When training a deep learning model, the optimization process may encounter steep valleys, flat regions, or saddle points in the loss landscape. If the learning rate is too high, the optimization process may oscillate around these regions, making it difficult for the model to converge to the optimal solution. By decaying the learning rate gradually, the optimization process becomes smoother, allowing the model to navigate these challenging areas and converge more effectively.

2. **Stability**: A high learning rate can cause large parameter updates, leading to overshooting and instability in the optimization process. On the other hand, too small a learning rate can result in slow convergence and may get stuck in local minima. A learning rate scheduler adjusts the learning rate dynamically based on the training progress, striking a balance between fast initial progress and fine-tuning as the training proceeds.

3. **Generalization**: Gradually reducing the learning rate can help the model generalize better to unseen data. When the learning rate is high, the model might make large updates that memorize the training data too closely, leading to poor generalization on new data. With a decaying learning rate, the model learns to capture more meaningful patterns in the data, resulting in improved generalization performance.

Now, let's understand how a learning rate scheduler works:

A learning rate scheduler is a mechanism that automatically adjusts the learning rate during the training process based on a predefined schedule or rule. The scheduler monitors the training progress, typically based on the number of epochs or the number of steps, and updates the learning rate accordingly.

There are various types of learning rate schedulers, but one common type is the step-based scheduler. In this scheduler, the learning rate is decreased at specific step intervals. For example, after every 'N' epochs or 'M' steps, the learning rate is multiplied by a decay factor (usually a value less than 1) to reduce its value. This process continues throughout the training process.

Here's a step-based learning rate scheduler in pseudocode:

```
initial_learning_rate = 0.1
decay_factor = 0.1
step_size = 10  # Adjust the learning rate every 10 epochs

optimizer = torch.optim.SGD(model.parameters(), lr=initial_learning_rate)

for epoch in range(num_epochs):
    # Train the model for one epoch
    for batch in train_dataloader:
        # Perform forward and backward passes
        loss = model(batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    # Adjust the learning rate after each 'step_size' epochs
    if (epoch + 1) % step_size == 0:
        new_learning_rate = initial_learning_rate * decay_factor
        for param_group in optimizer.param_groups:
            param_group['lr'] = new_learning_rate
```
With this scheduler, the learning rate will decrease by a factor of 0.1 every 10 epochs.

Learning rate schedulers can be more complex, involving various strategies like using exponential or polynomial decay, warm-up stages, or adapting the learning rate based on performance metrics. These strategies aim to strike a balance between exploring the optimization landscape and refining the model's parameters for better convergence and generalization.

**----END---**

---

##GPU to speed up training

In [24]:
import torch

In [25]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [26]:
model.to(device)
print(device)

cpu


In [27]:
#import torch

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#print(torch.cuda.is_available())  # This should print True if CUDA is available.
#print(device)  # This will print "cuda" if CUDA is available, or "cpu" otherwise.


##Training the model

In [28]:
from tqdm.auto import tqdm

In [29]:
progress_bar = tqdm(range(num_training_steps))

  0%|          | 0/1377 [00:00<?, ?it/s]

In [30]:
model.train()
for epoch in range(num_epochs):
  for batch in train_dataloader:
    batch={k: v.to(device) for k,v in batch.items()}
    outputs = model(**batch)
    loss = outputs.loss
    loss.backward()

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)

##Evaluating the model

In [31]:
from datasets import load_metric

In [32]:
metric = load_metric("glue", "mrpc")

<ipython-input-32-0220e2770ceb>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "mrpc")


In [33]:
model.eval()
for batch in eval_dataloader:
  batch = {k: v.to(device) for k,v in batch.items()}
  with torch.no_grad():
    outputs=model(**batch)

  logits = outputs.logits
  predictions = torch.argmax(logits, dim=1)
  metric.add_batch(predictions=predictions,references=batch["labels"])

In [34]:
metric.compute()

{'accuracy': 0.6911764705882353, 'f1': 0.8055555555555556}